In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from gen_ai_hub.proxy import get_proxy_client
proxy_client = get_proxy_client(proxy_version="gen-ai-hub")
proxy_client.get_deployments()

[Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc5b752a11e5fa73', config_id='1066614d-388f-4f86-8dcb-e8e37ed1fbe1', config_name='gpt-4o', deployment_id='dc5b752a11e5fa73', model_name='gpt-4o', created_at=datetime.datetime(2024, 7, 15, 8, 6, 34, tzinfo=datetime.timezone.utc), additonal_parameters={}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d7cb2d1ac98ffd3d', config_id='d010c3d2-61f8-4cce-922f-747747986e0e', config_name='Gemini', deployment_id='d7cb2d1ac98ffd3d', model_name='gemini-1.0-pro', created_at=datetime.datetime(2024, 6, 28, 14, 24, 10, tzinfo=datetime.timezone.utc), additonal_parameters={}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc34e5da947ad1d4', config_id='d010c3d2-61f8-4cce-922f-747747986e0e', config_name='Gemini', dep

In [3]:
from gen_ai_hub.proxy.langchain.init_models import init_llm


model = init_llm("gpt-4o")

model.invoke("Are you there?").content

"Yes, I'm here! How can I assist you today?"

In [13]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(state_schema=MessagesState)

def call_llm(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages" : response}

graph_builder.add_node("chatbot", call_llm)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [19]:
from langchain_core.messages import HumanMessage
# config = {"configarable": {"thread_id" : "abc123"}}
config = {"configurable": {"thread_id": "abc123"}}

while True:
    query = input("User: \n")
    if query.lower() == "exit":
        print("Good Bye!!!")
        break
    input_messages = [HumanMessage(content=query)]
    print(graph.invoke({"messages":input_messages}, config=config))
    print("--" * 200)


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='8cbf9fa5-d987-4817-a3e3-d00dcb1f2295'), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='18d3b28f-8683-4db3-beb9-1c0858fd31cf'), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='90222359-9747-419d-8b9d-0393ec6591b8'), AIMessage(content='hehehe', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.10669069737195969, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.13568978011608124}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.028007585555315018, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.07807836681604385}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_sc

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised BadRequest: 400 POST https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d7cb2d1ac98ffd3d/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Unable to submit request because it must include at least one parts field, which describes the prompt input. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini.
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised BadRequest: 400 POST https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d7cb2d1ac98ffd3d/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Unable to submit request because it must include at least one parts field, which describes the prompt input. Learn more: http

KeyboardInterrupt: 